## 토픽 모델링
* 실습을 위해 pyLDAvis 설치가 필요합니다. 
* colab사용시 설치 후에도 제대로 동작하지 않거나 오류가 나면 런타임을 재실행 해주세요.

In [1]:
# !pip install -U -q pyLDAvis

In [2]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

## 라이브러리 로드

In [3]:
# 필요 라이브러리를 로드
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## 데이터 로드

In [4]:
df = pd.read_csv("https://bit.ly/seoul-120-text-csv")
df.shape

(2645, 5)

In [5]:
# 결측치가 있다면 제거
df = df.dropna()
df.shape

(2645, 5)

In [6]:
# 결측치를 확인
df.isnull().sum()

번호      0
분류      0
제목      0
내용      0
내용번호    0
dtype: int64

## 문서 만들기
* 제목과 내용을 함께 사용

In [7]:
df["문서"] = df["제목"] + " " + df["내용"]

## 벡터화

* [Bag-of-words model - Wikipedia](https://en.wikipedia.org/wiki/Bag-of-words_model)


## CountVectorizer

* analyzer : 단어, 문자 단위의 벡터화 방법 정의
* ngram_range : BOW 단위 수 (1, 3) 이라면 1개~3개까지 토큰을 묶어서 벡터화
* max_df : 어휘를 작성할 때 문서 빈도가 주어진 임계값보다 높은 용어(말뭉치 관련 불용어)는 제외 (기본값=1.0)
    * max_df = 0.90 : 문서의 90% 이상에 나타나는 단어 제외
    * max_df = 10 : 10개 이상의 문서에 나타나는 단어 제외
* min_df : 어휘를 작성할 때 문서 빈도가 주어진 임계값보다 낮은 용어는 제외합니다. 컷오프라고도 합니다.(기본값=1.0)
    * min_df = 0.01 : 문서의 1% 미만으로 나타나는 단어 제외
    * min_df = 10 : 문서에 10개 미만으로 나타나는 단어 제외
* stop_words : 불용어 정의
* API Document: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [8]:
# 단어들의 출현 빈도(frequency)로 여러 문서들을 벡터화하기 위해 CountVectorizer 사용
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words=["돋움", "경우", "또는"])

### 참고: fit, transform, fit_transfrom의 차이점
- fit(): 원시 문서에 있는 모든 토큰의 어휘 사전을 배운다
- transform(): 문서를 문서 용어 매트릭스로 변환, transform 이후엔 매트릭스로 변환되어 숫자형태로 변경
- fit_transform(): 어휘 사전을 배우고 문서 용어 매트릭스를 반환, fit 다음에 변환이 오는 것과 동일하지만 더 효율적으로 구현

* API Document: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.fit_transform

In [9]:
# fit_transform을 사용하여 문장에서 노출되는 feature(특징이 될만한 단어) 수를 합한 변수 Document Term Matrix(이하 dtm)를 생성
dtm_cv = cv.fit_transform(df["문서"])

In [10]:
# cv.vocabulary_ 를 봅니다.
# cv.vocabulary_

In [11]:
cv_cols = cv.get_feature_names()

In [12]:
# 각 row에서 전체 단어가방에 있는 어휘에서 등장하는 단어에 대한 one-hot-vector를 확인
# toarray()로 희소 행렬(sparse matrix, 행렬의 값이 대부분 '0'인 행렬)을 NumPy array 배열로 변환하여 값을 확인

pd.DataFrame(dtm_cv.toarray(), columns=cv_cols).sum().sort_values()

03월       1
용액속에      1
용액을       1
용액이       1
용어        1
       ... 
대한      394
서울시     578
어떻게     597
있습니다    685
있는      718
Length: 56651, dtype: int64

## BOW<sup>bag of word</sup> 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)

* API documentation: https://pyldavis.readthedocs.io/en/latest/modules/API.html

In [13]:
# 정답인 '분류'의 유일한 값을 확인하여 주제 수를 확인
df["분류"].value_counts()

행정        1098
경제         823
복지         217
환경         124
주택도시계획     110
문화관광        96
교통          90
안전          51
건강          23
여성가족        13
Name: 분류, dtype: int64

In [14]:
# 주어진 문서에 대하여 각 문서에 어떤 주제들이 존재하는지를 확인하는 잠재 디리클레 분석(LDA)을 불러옴
# n_components에 넣을 하이퍼파라미터 NUM_TOPICS로 주제수를 설정(기본값=10)
# max_iter는 훈련 데이터(epoch라고도 함)에 대한 최대 패스 수(기본값=10)

from sklearn.decomposition import LatentDirichletAllocation

NUM_TOPICS = 10
LDA_model = LatentDirichletAllocation(n_components=NUM_TOPICS, random_state=42)

In [15]:
# LDA_model 에 dtm_cv 를 넣어 학습
LDA_model.fit(dtm_cv)

LatentDirichletAllocation(random_state=42)

### pyLDAvis

In [16]:
# !pip install -U -q pyLDAvis

In [17]:
# 토픽 모델링에 이용되는 LDA 모델의 학습 결과를 시각화하는 Python 라이브러리인 pyLDAvis를 불러옴
# mds(Multi-Dimensional Scaling)는 데이터 포인트 간의 거리를 보존하면서 차원을 축소하는 기법
# t-SNE(t-Stochastic Neighbor Embedding)은 고차원 데이터를 특히 2, 3차원 등으로 줄여 가시화하는데에 유용

import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(LDA_model, dtm_cv, cv, mds='tsne')

/Users/corazzon/opt/anaconda3/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
3      32.420952   8.115230       1        1  13.259921
7       0.648955  71.707100       2        1  11.412742
2      71.019684   9.971006       3        1  11.228595
8       3.785981  33.185135       4        1  10.144832
9     -42.666500  -9.222070       5        1  10.043109
0      -3.621689 -43.526516       6        1   9.877599
4      -3.632402  -4.171708       7        1   9.539921
5     -36.584400  35.804413       8        1   9.012524
6      44.391380  50.615383       9        1   8.086331
1      40.235809 -31.591665      10        1   7.394426, topic_info=        Term        Freq       Total Category  logprob  loglift
119      amp   63.000000   63.000000  Default  30.0000  30.0000
39480   자본금은   52.000000   52.000000  Default  29.0000  29.0000
36388     의한  205.000000  205.000000  Default  28.0000  28.0000
8795   기술능력과   34.000000   34.000000  Default  27.0000  27.0000
31204    어떻게  441.000000  441.000000  Default  26.0000  26.0000
...      ...         ...         ...      ...      ...      ...
53924    합니다   14.178559  196.194444  Topic10  -6.6828  -0.0229
3575    경우에는   13.916488  192.141237  Topic10  -6.7015  -0.0207
9077      기타   12.125611  120.348058  Topic10  -6.8392   0.3094
27828     시간   12.035864  115.813247  Topic10  -6.8467   0.3404
304       가능   12.074554  226.443098  Topic10  -6.8435  -0.3269

[749 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
119        3  0.980964      amp
119        5  0.015822      amp
121        3  0.960074     apos
304        1  0.114819       가능
304        2  0.128067       가능
...      ...       ...      ...
56176     10  0.055042       회의
56308      7  0.924373  후계농업경영인
56375      1  0.208700       훈련
56375      5  0.730451       훈련
56375      7  0.104350       훈련

[1810 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 8, 3, 9, 10, 1, 5, 6, 7, 2])

## TF-IDF(Term Frequency - Inverse Document Frequency)

## TfidfVectorizer

TF-IDF 인코딩은 단어를 갯수 그대로 카운트하지 않고 모든 문서에 공통적으로 들어있는 단어(낮은 구별력)의 경우 가중치를 축소하는 방법

매개변수
* norm='l2' 각 문서의 피처 벡터를 어떻게 벡터 정규화 할지 정한다. 
    - L2 : 벡터의 각 원소의 제곱의 합이 1이 되도록 만드는 것이고 기본 값
    - L1 : 벡터의 각 원소의 절댓값의 합이 1이 되도록 크기를 조절
* smooth_idf=False
    - 피처를 만들 때 0으로 나오는 항목에 대해 작은 값을 더해서(스무딩을 해서) 피처를 만들지 아니면 그냥 생성할지를 결정
* sublinear_tf=False
* use_idf=True
    - TF-IDF를 사용해 피처를 만들 것인지 아니면 단어 빈도 자체를 사용할 것인지 여부
* API Document: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer


In [18]:
# TF-IDF 방식으로 단어의 가중치를 조정한 BOW 인코딩하여 벡터화하기 위해 TfidfVectorizer를 사용

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=["돋움", "경우", "또는", "있습니다", "있는", "합니다"])
tfidf

TfidfVectorizer(stop_words=['돋움', '경우', '또는', '있습니다', '있는', '합니다'])

In [19]:
# 문장에서 노출되는 feature(특징이 될만한 단어) 수를 합한 변수 Document Term Matrix(이하 dtm)를 생성
dtm_tfidf = tfidf.fit_transform(df["문서"])

In [20]:
# tfidf.vocabulary_
cols_tfidf = tfidf.get_feature_names()

In [21]:
# dtm_tf를 axis=0(수직 방향으로) 기준으로 합계를 낸 dist 변수를 생성
# dist 변수를 vocabulary_ 순으로 정렬하여 비율을 확인
dist = np.sum(dtm_tfidf, axis=0)
pd.DataFrame(dist, columns=cols_tfidf).T.sort_values(by=0).tail(10)

,0
의한,15.021840
무엇입니까,15.270257
이상,15.577954
관한,16.593598
무엇인가요,16.650743
따라,16.652594
대한,18.866037
있나요,19.707343
서울시,22.586695
어떻게,37.924574


In [22]:
# 각 row에서 전체 단어가방에 있는 어휘에서 등장하는 단어에 대한 가중치를 적용한 vector를 확인
# toarray()로 희소 행렬(sparse matrix, 행렬의 값이 대부분 '0'인 행렬)을 NumPy array 배열로 변환하여 값을 확인
pd.DataFrame(dtm_tfidf.toarray(), columns=cols_tfidf)

,03월,08년,10,100명이상인,100세가,10만원,10만원상당,10명이고,10인승,10인의,...,힐링프로그램을,힐링하는,힐스테이트,힘들,힘들경우,힘들고,힘쓰고있습니다,힘쓴다,힘을,힘이
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2640,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2641,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## TF-IDF 잠재 디리클레 할당(LDA)

In [23]:
# 주어진 문서에 대하여 각 문서에 어떤 주제들이 존재하는지를 확인하는 잠재 디리클레 분석(LDA)을 사용
# n_components에 넣을 하이퍼파라미터 NUM_TOPICS로 주제수를 설정(기본값=10)
# max_iter는 훈련 데이터(epoch라고도 함)에 대한 최대 패스 수(기본값=10)

NUM_TOPICS = 10 
LDA_model = LatentDirichletAllocation(n_components=NUM_TOPICS, 
                                      max_iter=30, 
                                      random_state=42)

In [24]:
# dtm_tfidf 를 LDA_model로 학습
LDA_model.fit(dtm_tfidf)

LatentDirichletAllocation(max_iter=30, random_state=42)

In [25]:
# 토픽 모델링에 이용되는 LDA 모델의 학습 결과를 시각화하는 Python 라이브러리인 pyLDAvis를 사용
# mds(Multi-Dimensional Scaling)는 데이터 포인트 간의 거리를 보존하면서 차원을 축소하는 기법
# t-SNE(t-Stochastic Neighbor Embedding)은 고차원 데이터를 특히 2, 3차원 등으로 줄여 가시화하는데에 유용

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(LDA_model, dtm=dtm_tfidf, vectorizer=tfidf, mds='tsne')

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
2      32.420952   8.115230       1        1  13.379687
8       0.648955  71.707100       2        1  11.603278
1      71.019684   9.971006       3        1  10.721346
6       3.785981  33.185135       4        1  10.146669
5     -42.666500  -9.222070       5        1   9.686810
4      -3.621689 -43.526516       6        1   9.308106
3      -3.632402  -4.171708       7        1   9.278252
0     -36.584400  35.804413       8        1   8.740215
7      44.391380  50.615383       9        1   8.696668
9      40.235809 -31.591665      10        1   8.438969, topic_info=        Term      Freq     Total Category  logprob  loglift
39478   자본금은  2.000000  2.000000  Default  30.0000  30.0000
8795   기술능력과  1.000000  1.000000  Default  29.0000  29.0000
48611    청소년  1.000000  1.000000  Default  28.0000  28.0000
2408   건설기술자  1.000000  1.000000  Default  27.0000  27.0000
36726   이상이고  1.000000  1.000000  Default  26.0000  26.0000
...      ...       ...       ...      ...      ...      ...
16380     문의  0.201512  2.740665  Topic10  -8.6826  -0.1378
14923   말합니다  0.196629  2.399825  Topic10  -8.7072  -0.0295
16122  무엇인가요  0.204140  3.557390  Topic10  -8.6697  -0.3857
14075     등의  0.194355  2.555684  Topic10  -8.7188  -0.1041
14074     등을  0.191920  3.001739  Topic10  -8.7314  -0.2776

[674 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
304        1  0.370575     가능
2408       2  0.943309  건설기술자
4859       1  0.472743    공무원
5683       1  0.393888     관련
5761       2  1.227986  관련종목의
...      ...       ...    ...
52548      1  0.641556   프로그램
52837      2  0.311276    하나요
52908      3  0.318458     하는
55325      1  0.294068   홈페이지
55625      1  0.421737     확인

[75 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 9, 2, 7, 6, 5, 4, 1, 8, 10])

## 코사인 유사도
* API Document: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_simple_pair = cosine_similarity(dtm_tfidf[0] , dtm_tfidf)
result_list = similarity_simple_pair.tolist()[0]

In [28]:
df["유사도"] = result_list
df[["분류", "제목", "유사도"]].sort_values(by="유사도", ascending=False).head(10)

,분류,제목,유사도
0,복지,아빠 육아휴직 장려금,1.000000
1772,경제,도시계획시설부지 재결신청 이후 진행단계는 어떤 과정을 거칩니까?,0.057158
850,경제,주민대표회의 구성원 몇명입니까?,0.056152
539,행정,행려자도 아니고 시설수용자도 아닌 사람이 살고 있던 비닐하우스에서 화상을 입었습니다...,0.051952
3,복지,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",0.048404
155,경제,[농업기술센터] 후계농업경영인 선정 및 청년창업형 후계농업경영인 신청 안내,0.046280
35,행정,[시ㆍ구정외 타기관 관련 상담] 고용노동부 [일자리 안정자금],0.046037
141,경제,[농업기술센터] 도시농업전문가양성교육 신청,0.043873
174,행정,찾아가는 아버지교실,0.043034
233,경제,[농업기술센터] 귀농창업 평일반 교육 신청,0.041945
